In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import cv2
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import glob
from PIL import Image
from collections import defaultdict
import math
from copy import deepcopy
import pandas as pd
import struct, os
import re, numpy as np
# from skimage import transform
import itertools, operator
import pickle
from torch.optim.lr_scheduler import _LRScheduler
from nltk.corpus import wordnet as wn

import sys
import xml.etree.ElementTree as ET

import numpy as np
from PIL import Image
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.datasets as dsets
import torchvision
import torchvision.transforms as transforms
import torch.utils.data as td 
import torchvision as tv
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, ConcatDataset

import Data
import build
from build import Yolov2
import bbox

import nntool as nt 

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

## Load dataset and settings

In [ ]:
dataset_root_dir = '/datasets/ee285f-public/PascalVOC2012'
batch_size = 32
meta = defaultdict()
meta['anchors'] = 5
meta['classes'] = 20
meta['batch_size'] = batch_size
meta['threshold'] = 0.6
meta['anchor_bias'] = np.array([1.08,1.19,  3.42,4.41,  6.63,11.38,  9.42,5.11,  16.62,10.52])
meta['scale_no_obj'] = 1
meta['scale_coords'] = 1
meta['scale_class'] = 1
meta['scale_obj']  = 5
meta['iteration'] = 0 

In [ ]:
train_set, test_set = Data.getdata(dataset_root_dir)
meta['train_samples'] = len(train_set)
meta['iterations_per_epoch'] = meta['train_samples']/batch_size

bbox.draw_bbox_torch(train_set[0])

## Set up stats manager

In [ ]:
class YOLOStatsManager(nt.StatsManager):
    def __init__(self):
        super(YOLOStatsManager, self).__init__()

    def init(self):
        super(YOLOStatsManager, self).init()
        self.AP = 0
        self.n = 1e-6
        
    def accumulate(self, loss, x, y, d, mode): 
        super(YOLOStatsManager, self).accumulate(loss)
        if mode == 'test':
            AP, n = self.mAP(x, y, d)
            self.AP += AP
            self.n += n
        
    def summarize(self):
        loss = super(YOLOStatsManager, self).summarize() 
        return {'loss': loss, 'mAP': self.AP/self.n}

    def mAP(self, y, ground_truths, n_true, iou_thresh = 0.5):
        
        nms_output = bbox.get_nms_detections(y, 0.24, 0.3)
        n_truths = n_true.numpy()
        N = ground_truths.size(0)
    
        mean_avg_precision = Variable(torch.FloatTensor([0]))
        if torch.cuda.is_available():
            mean_avg_precision = mean_avg_precision.cuda()

        for batch in range(int(N)):
            category_map = defaultdict(lambda: defaultdict(lambda: torch.FloatTensor()))
        
            if n_truths[batch] == 0:
                continue

            ground_truth = ground_truths[batch, :n_truths[batch]]
            for gt in ground_truth:
                gt_class = gt[0].int().data[0]
                t1 = category_map[gt_class]['ground_truth']
                if len(t1.size()) == 0:
                    t1 = gt[1:].unsqueeze(0)
                else:
                    t1 = torch.cat([t1, gt[1:].unsqueeze(0)], 0)
                category_map[gt_class]['ground_truth'] = t1
            
            nms_boxes = nms_output[batch]
            if len(nms_boxes.size()) == 0:
                continue

            for box in nms_boxes:
                class_id = (torch.max(box[5:], 0)[1]).int().data[0]
                t2 = category_map[class_id]['prediction']
                if len(t2.size()) == 0:
                    t2 = box[:5].unsqueeze(0)
                else:
                    t2 = torch.cat([t2, box[:5].unsqueeze(0)], 0)
                category_map[class_id]['prediction'] = t2
            cat_ids = category_map.keys()
#         return category_map
            mean_avg_precision += torch.mean(torch.cat([calc_map(category_map[cat_id], iou_thresh) for cat_id in cat_ids], 0 ))
        return mean_avg_precision, N

## Exp1: train last 4 layers only

In [ ]:
net = Yolov2()
net = build.load_pretrained_weights(net)
build.model_freeze_upto(net, 36)
for name, param in net.named_parameters():
    print(name, param.size(), param.requires_grad)

In [ ]:
#!rm -r yolo_voc1

lr = 0.00025
net = net.to(device)
adam = torch.optim.Adam(net.parameters(), lr=lr)
stats_manager = YOLOStatsManager()
exp1 = nt.Experiment(net, train_set, test_set, adam, stats_manager, meta, output_dir="yolo_voc1", 
                     batch_size = batch_size, perform_validation_during_training=False)

In [ ]:
def plot(exp, fig, axes):
    axes.clear()
    axes.plot([exp.history[k]['loss'] for k in range(len(exp.history))], label="training loss")
#     axes.plot([exp.history[k][1]['loss'] for k in range(exp.epoch)], label="evaluation loss")
    plt.xticks([18,36,54,72], ['epoch 1', 'epoch 2', 'epoch 3', 'epoch 4'], rotation=45)    
    plt.tight_layout()
    fig.canvas.draw()
    axes.set_ylim([0,50])
    axes.legend()

fig, axes = plt.subplots(ncols=1, figsize=(12, 5))
exp1.run(num_epochs=4, plot=lambda exp: plot(exp, fig=fig, axes=axes))

In [ ]:
#evaluate
test_loader = td.DataLoader(test_set,
                                     batch_size=batch_size,
                                     shuffle=True,
                                     drop_last=True,
                                     pin_memory=True)
for i, test_data in enumerate(test_loader):
    test_images = Variable(test_data["image"], requires_grad=True).cuda().float()
    test_labels = Variable(test_data["bboxes"], requires_grad=False).cuda().float()
    test_n_true = test_data["n_true"]
    break

net = net.cuda()
a=net.forward(test_images)
nnms = bbox.get_nms_detections(a, 0.24, 0.3)
bbox.draw_bbox_torch({"image": test_images[0].cpu().data, "bboxes": test_labels[0].cpu().data})
bbox.draw_bbox_nms(test_images[0].cpu(), nnms[0].cpu())


## Exp2: Fine-tuning: train all layers

In [ ]:
net = Yolov2()
net = build.load_pretrained_weights(net)
build.model_freeze_upto(net, 0)
for name, param in net.named_parameters():
    print(name, param.size(), param.requires_grad)

In [ ]:
#!rm -r yolo_voc2

lr = 0.00025
net = net.to(device)
adam = torch.optim.Adam(net.parameters(), lr=lr)
stats_manager = YOLOStatsManager()
exp2 = nt.Experiment(net, train_set, test_set, adam, stats_manager, meta, output_dir="yolo_voc2", 
                     batch_size = batch_size, perform_validation_during_training=False)

In [ ]:
def plot(exp, fig, axes):
    axes.clear()
    axes.plot([exp.history[k]['loss'] for k in range(len(exp.history))], label="training loss")
#     axes.plot([exp.history[k][1]['loss'] for k in range(exp.epoch)], label="evaluation loss")
    plt.xticks([18,36,54,72], ['epoch 1', 'epoch 2', 'epoch 3', 'epoch 4'], rotation=45)    
    plt.tight_layout()
    fig.canvas.draw()
    axes.set_ylim([0,50])
    axes.legend()

fig, axes = plt.subplots(ncols=1, figsize=(12, 5))
exp2.run(num_epochs=4, plot=lambda exp: plot(exp, fig=fig, axes=axes))

## Exp3: Cosine annealing learning rate

In [ ]:
class CosineAnnealing(_LRScheduler):

    def __init__(self, optimizer, T_max, eta_min=0, last_epoch=-1):
        self.T_max = T_max
        self.eta_min = eta_min
        super(CosineAnnealing, self).__init__(optimizer, last_epoch)

    def get_lr(self):
        return [self.eta_min + (base_lr - self.eta_min) *
                (1 + math.cos(math.pi * self.last_epoch / self.T_max)) / 2
                for base_lr in self.base_lrs]

In [ ]:
net = Yolov2()
net = build.load_pretrained_weights(net)
build.model_freeze_upto(net, 36)
for name, param in net.named_parameters():
    print(name, param.size(), param.requires_grad)

In [ ]:
#!rm -r yolo_voc3

lr = 0.00025
net = net.to(device)
adam = torch.optim.Adam(net.parameters(), lr=lr)
ca = CosineAnnealing(adam, 300)
stats_manager = YOLOStatsManager()
exp3 = nt.Experiment(net, train_set, test_set, ca, stats_manager, meta, output_dir="yolo_voc3", 
                     batch_size = batch_size, perform_validation_during_training=False)